In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### Pretrain datasets

In [2]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.02s/it]

CPU times: user 38.6 s, sys: 42.7 s, total: 1min 21s
Wall time: 56.1 s


In [3]:
from sklearn.model_selection import train_test_split

df_train_trans, df_valid_trans = train_test_split(df_trans, test_size = 0.1, random_state = 42)
df_train_trans = df_train_trans.to_dict(orient='records')
df_valid_trans = df_valid_trans.to_dict(orient='records')

In [4]:
len(df_train_trans), len(df_valid_trans)

(373785, 41532)

## Train MLM

### Model definition

In [5]:
import torch
import pytorch_lightning as pl
from functools import partial
from ptls.nn import TrxEncoder, TransformerEncoder
from ptls.nn import PBLinear, PBL2Norm, PBLayerNorm
from ptls.frames.bert import MLMPretrainModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'request_cnt': 'identity'},
    embeddings={ 
        'part_of_day': {'in': 4, 'out': 1},
        'url_host': {'in': 132025, 'out': 512}
    }
)

trx_encoder = TrxEncoder(**trx_encoder_params)
trx_encoder = torch.nn.Sequential(trx_encoder, PBLinear(trx_encoder.output_size, 256), PBL2Norm())

seq_encoder = TransformerEncoder(input_size=256, is_reduce_sequence=True)

model = MLMPretrainModule(trx_encoder=trx_encoder, seq_encoder=seq_encoder, hidden_size=256, total_steps=100000)

### Dataloader

In [6]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.frames.bert import MlmDataset


train_ds = MemoryMapDataset(data=df_train_trans)
valid_ds = MemoryMapDataset(data=df_valid_trans)

train_data=MlmDataset(train_ds, min_len=25, max_len=256)
valid_data=MlmDataset(valid_ds, min_len=25, max_len=256)

dl = PtlsDataModule(train_data=train_data, valid_data=valid_data, train_num_workers=16, train_batch_size=256)

### Trainer

In [7]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_steps=50000,
    limit_val_batches=100,
    gpus=[0],
    enable_progress_bar=False,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, dl)
print(trainer.logged_metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


logger.version = 19



  | Name            | Type               | Params
-------------------------------------------------------
0 | trx_encoder     | Sequential         | 67.7 M
1 | seq_encoder     | TransformerEncoder | 2.4 M 
2 | fn_norm_predict | PBShell            | 0     
3 | loss_fn         | QuerySoftmaxLoss   | 0     
4 | train_mlm_loss  | MeanMetric         | 0     
5 | valid_mlm_loss  | MeanMetric         | 0     
-------------------------------------------------------
70.1 M    Trainable params
0         Non-trainable params
70.1 M    Total params
280.417   Total estimated model params size (MB)


In [ ]:
torch.save(model.state_dict(), "mlm-emb.pt")

# Inference

In [ ]:
model.load_state_dict(torch.load("mlm-emb.pt"))

In [ ]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

In [ ]:
%%time
import tqdm
from ptls.data_load.datasets import inference_data_loader
import numpy as np

def pooling_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            x = model.trx_encoder(batch.to(device)).payload
            out_max = torch.max(x, dim=1)[0]
            out_min = torch.min(x, dim=1)[0]
            out_mean = torch.mean(x, dim=1)
            out_std = torch.std(x, dim=1)
            features = torch.cat([out_max, out_min, out_mean, out_std], dim=1)      
            X += [features]
    return X

def embed_inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            z = model.trx_encoder(batch.to(device))
            features = model.seq_encoder(z)
            X += [features]
    return X

dl = inference_data_loader(df_trans.to_dict(orient='records'), num_workers=0, batch_size=128)
#X_mlm = torch.vstack(embed_inference(model, dl, )).cpu().numpy()
X_pool = torch.vstack(pooling_inference(model, dl, )).cpu().numpy()
#X_embeds = np.concatenate([X_mlm, X_pool], axis=1)
X_embeds = X_pool

df_embeds = pd.DataFrame(X_embeds, columns=[f"mlm_{e}" for e in range(X_embeds.shape[1])])
df_embeds['user_id'] = df_trans['user_id']
df_embeds.to_csv('./data/mlm_512.csv', index=False)

## Downstream

In [ ]:
%%time

import bisect
import numpy as np

df_embeds = pd.read_csv('./data/mlm_512.csv')
df_public = pq.read_table('data/public_train.pqt').to_pandas().sort_values(by='user_id')

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

y_age = df_public['age']
y_age = np.array(list(map(age_bucket, y_age)))
y_gender = np.array(df_public['is_male'])

X = df_public
X = X.merge(df_embeds, on="user_id", how='inner')
del X['user_id'], X['age'], X['is_male']

## Gender

In [ ]:
# %%time

from catboost import CatBoostClassifier, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
not_na_gender = (y_gender != 'NA') & (y_gender != None)
x_train, x_test_gender, y_train, y_test_gender = train_test_split(X[not_na_gender], y_gender[not_na_gender], test_size = 0.1, random_state = 42)

clf_gender = CatBoostClassifier(
    iterations=1000,
    custom_metric=[metrics.AUC()],
    use_best_model=True,
    random_seed=42)
clf_gender.fit(x_train, y_train, metric_period=100, eval_set=(x_test_gender, y_test_gender))

In [23]:
print(f'GINI по полу {2 * roc_auc_score(y_test_gender, clf_gender.predict_proba(x_test_gender)[:,1]) - 1:2.3f}')

GINI по полу 0.694


# Age

In [ ]:
%%time

from sklearn.metrics import classification_report

not_na_age = ~np.isnan(y_age)
x_train, x_test_age, y_train, y_test_age = train_test_split(X[not_na_age], y_age[not_na_age], test_size = 0.1, random_state = 42)

clf_age = CatBoostClassifier(iterations=1000,
    custom_metric=[metrics.Accuracy()],
    use_best_model=True,
    random_seed=42)
clf_age.fit(x_train, y_train, metric_period=100, eval_set=(x_test_age, y_test_age))

In [22]:
print(classification_report(y_test_age, clf_age.predict(x_test_age), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

              precision    recall  f1-score   support

         <18       0.00      0.00      0.00       109
       18-25       0.53      0.35      0.42      3238
       25-34       0.50      0.64      0.56      8863
       35-44       0.41      0.51      0.45      7773
       45-54       0.38      0.23      0.29      4218
       55-65       0.39      0.22      0.29      2254
         65+       0.38      0.02      0.03       545

    accuracy                           0.45     27000
   macro avg       0.37      0.28      0.29     27000
weighted avg       0.44      0.45      0.43     27000



In [20]:
0.752 + 2*0.46

1.6720000000000002

In [21]:
0.760 + 2*0.47

1.7